In [1]:
from evaluator import *
import pandas as pd
import sklearn.preprocessing as pp

In [49]:
def make_csv():
    ir = ImageRecognition()
    #  ir.load_processed_data('data/data_image_100.pkl')
    #  load_metadata('metadata_02-20-2017_02:10:32.pkl')
    ir.load_processed_data('data/data_csv_only.pkl')
    test = ir.data[:, :-99]
    label = ir.data[:, -99:]
    metadata = dict(
        test=test,
        test_labels=label,
        id=ir.identifier,
        one_hot_names=ir.one_hot_names,
        n_input=ir.n_input,
        model_name = 'mlp_02-23-2017_06:46:46-199',
        weight_dir="weight/",
        prediction_dir="predictions/",
        tensor_name="output/activation:0",
        input_x_tensor_name="input_1/x-input:0",
        input_y_tensor_name="input_1/y-input:0",
        learning_rate=0.01,
        debug=True)

#     load_session_and_save_prediction(**metadata)
#     pred = load_predictions(metadata['prediction_dir'] + metadata['model_name'])
#     return pred
    test_csv = pd.read_csv('csv/test.csv')
    test_data = test_csv.drop(['id'], axis=1).as_matrix()
    n_input = test_data.shape[1]
    identifier = test_csv['id'].as_matrix()
    
    sess = load_session(metadata['weight_dir']+metadata['model_name'])
    y_ = get_tensor(sess, metadata['tensor_name'])
    prediction = predict(sess, y_, test_data)
    
    #scale
    pred_0_to_1 = pp.minmax_scale(prediction, axis=1)
    pred = pred_0_to_1/np.sum(pred_0_to_1, axis=1, keepdims=True)
    
    df = pd.DataFrame(data=pred, index=identifier,columns=ir.one_hot_names)
    df.index.name='id'
    return df,pred
    

In [50]:
df,pred = make_csv()


In [51]:
pred[0]


array([ 0.00860966,  0.00722096,  0.0057887 ,  0.02284247,  0.00544109,
        0.00629637,  0.01195056,  0.        ,  0.00746335,  0.00672085,
        0.00977054,  0.00797963,  0.01107451,  0.00758527,  0.00448637,
        0.00891771,  0.00845922,  0.00795986,  0.01229754,  0.01151778,
        0.00870499,  0.00330319,  0.01043391,  0.00574421,  0.00987229,
        0.00520509,  0.01372601,  0.00992724,  0.01190429,  0.01050903,
        0.00675416,  0.0129385 ,  0.01395004,  0.00706421,  0.01094361,
        0.00604369,  0.01111644,  0.00524123,  0.00715184,  0.00776398,
        0.00449313,  0.01215876,  0.01015238,  0.01107574,  0.00841944,
        0.01397986,  0.01391597,  0.0107188 ,  0.00576911,  0.00767963,
        0.01657474,  0.02078981,  0.01261393,  0.01352408,  0.00909377,
        0.02132402,  0.01335472,  0.00561297,  0.01290453,  0.01086604,
        0.01059685,  0.01027219,  0.00794293,  0.00957432,  0.01329466,
        0.01225146,  0.00552653,  0.00939236,  0.01343192,  0.01

In [41]:
df.to_csv('test_submission.csv')
